### Applying function calling using Langchain and OpenAI
- make sure to use the upgraded langchain and pydantic version.
- if u refer to DeepLearning.ai course on function & tools, that pydantic version is old.
- this is a code on how function calling behaves and how to use it.

py311

In [148]:
from langchain.schema.agent import AgentFinish
from langchain.chat_models import ChatOpenAI
from typing import List
from pydantic.v1 import BaseModel, Field
from langchain.agents import tool
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.schema.agent import AgentFinish

In [150]:
import langchain
langchain.__version__

'0.0.350'

In [151]:
import pydantic
pydantic.__version__

'2.5.2'

In [152]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [153]:
from typing import List
from pydantic.v1 import BaseModel, Field

### Pydantic function

pydantic function and it's description is needed to guide the LLM model to choose which function to call.

In [154]:
class AnimalSearch(BaseModel):
    """Call this to get the animal"""
    animal: str = Field(description="animal that is mentioned")

In [155]:
class WeatherSearch(BaseModel):
    """Call this to get the weather at that location"""
    location: str = Field(description="location to get weather for")

### Tool Function

tool function is the function to run when LLM model has decided on which function to call. 
but this function needs to be triggered by us. (see the end of the notebook)

In [184]:
from langchain.agents import tool
@tool(args_schema=WeatherSearch)
def search_weather(location: str) -> str:
    """Run weather search."""
    return "i am weather bob" #just a simple one. just to test

In [185]:
@tool(args_schema=AnimalSearch)
def search_animal(animal: str) -> str:
    """Run animal search."""
    return "i am a pink panther" #just a simple one. just to test

In [183]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
#weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [159]:
tools = [search_weather, search_animal]

The LLM model needs to be bind with functions. This indicates that it can utilize the function. 

In [182]:
from langchain.tools.render import format_tool_to_openai_function
#from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

functions = [format_tool_to_openai_function(f) for f in tools]
model_bind = ChatOpenAI(temperature=0).bind(functions=functions)

In [177]:
#you can dump this inside OpenAI Assistant API for function calling purpose
print(functions)

[{'name': 'search_weather', 'description': 'search_weather(location: str) -> str - Run weather search.', 'parameters': {'title': 'WeatherSearch', 'description': 'Call this to get the weather at that location', 'type': 'object', 'properties': {'location': {'title': 'Location', 'description': 'location to get weather for', 'type': 'string'}}, 'required': ['location']}}, {'name': 'search_animal', 'description': 'search_animal(animal: str) -> str - Run animal search.', 'parameters': {'title': 'AnimalSearch', 'description': 'Call this to get the animal', 'type': 'object', 'properties': {'animal': {'title': 'Animal', 'description': 'animal that is mentioned', 'type': 'string'}}, 'required': ['animal']}}]


In [178]:
#test whether it triggers 'search_weather' or 'search_animal'
model_bind.invoke("what is the weather in Nilai today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "location": "Nilai"\n}', 'name': 'search_weather'}})

In [179]:
#test whether it triggers 'search_weather' or 'search_animal'
model_bind.invoke("what is that rhino doing in Nilai today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "animal": "rhino"\n}', 'name': 'search_animal'}})

In [163]:
#example of forcing a specific function call
weather_function = convert_pydantic_to_openai_function(WeatherSearch)
model_bind.invoke("what is the animal in Nilai today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "location": "Nilai"\n}', 'name': 'WeatherSearch'}})

In [164]:
model_bind.invoke("I like to eat giraffe")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "animal": "giraffe"\n}', 'name': 'search_animal'}})

# Chain - LCEL the prompt, model

In [165]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    ("user", "{query}"),
])
chain = prompt | model_bind | OpenAIFunctionsAgentOutputParser()

In [166]:
result = chain.invoke({"query": "I am thinking about rabbit"})
result

AgentActionMessageLog(tool='search_animal', tool_input={'animal': 'rabbit'}, log="\nInvoking: `search_animal` with `{'animal': 'rabbit'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "animal": "rabbit"\n}', 'name': 'search_animal'}})])

In [167]:
result.tool_input

{'animal': 'rabbit'}

In [168]:
search_animal(result.tool_input)

'i am a pink panther'

In [169]:
search_animal("hippo")

'i am a pink panther'

- The above part shows that the LLM model wont run the function.
- When invoked, the result will show us what function is chosen, BUT it wont run it. 
- So, we need to write a code to input the "tool_input" (parameter) into the function to get the answer.
- To continue, see below.

# Final chain - LCEL the prompt, model, and route 

In [170]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_weather": search_weather, 
            "search_animal": search_animal,
        }
        return tools[result.tool].run(result.tool_input)

In [171]:
final_chain = prompt | model_bind | OpenAIFunctionsAgentOutputParser() | route

In [172]:
final_result = final_chain.invoke({"query": "find rabbit"})
final_result

'i am a pink panther'

In [173]:
final_result = final_chain.invoke({"query": "what is weather at Kuala Lumpur?"})
final_result

'i am weather bob'

So, it works. Bubye